In [ ]:
# --- Conexión serial y envío de estados al ESP32 ---
# Funciones para listar puertos, seleccionar uno y enviar los estados: 'sana', 'danada', 'defectosa'
try:
    import serial
    import serial.tools.list_ports as list_ports
except ImportError:
    import sys, subprocess
    print('pyserial no instalado. Instalando pyserial...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyserial'])
    import serial
    import serial.tools.list_ports as list_ports
import time

VALID_STATES = ("sana", "danada", "defectosa")


def listar_puertos():
    """Devuelve lista de puertos serial disponibles y los imprime."""
    ports = list(list_ports.comports())
    dispositivos = []
    if not ports:
        print("No se encontraron puertos serie disponibles.")
        return dispositivos
    for i, p in enumerate(ports):
        print(f"{i}: {p.device} - {p.description}")
        dispositivos.append(p.device)
    return dispositivos


def seleccionar_puerto(preferido: str = None):
    """Selecciona un puerto. Si 'preferido' está presente, lo devuelve; si no, devuelve el primero encontrado."""
    puertos = listar_puertos()
    if not puertos:
        return None
    if preferido and preferido in puertos:
        print(f"Usando puerto preferido: {preferido}")
        return preferido
    if len(puertos) == 1:
        print(f"Seleccionado puerto: {puertos[0]}")
        return puertos[0]
    print(f"Varios puertos disponibles, se selecciona por defecto: {puertos[0]}\nPara usar otro, llama seleccionar_puerto('/dev/ttyUSB1')")
    return puertos[0]


def send_state(port: str, state: str, baud: int = 115200, timeout: float = 1.0):
    """Envía un estado por serial y lee una respuesta opcional.

    Args:
        port: ruta del puerto (por ejemplo '/dev/ttyUSB0')
        state: uno de VALID_STATES
        baud: velocidad en baudios
        timeout: tiempo de espera en segundos

    Returns:
        La respuesta leída (string) o None si no hay respuesta.
    """
    if state not in VALID_STATES:
        raise ValueError(f"Estado inválido: {state}. Usa uno de: {VALID_STATES}")
    if port is None:
        raise RuntimeError("Puerto no especificado. Usa seleccionar_puerto() para obtener uno.")

    try:
        ser = serial.Serial(port, baudrate=baud, timeout=timeout)
    except Exception as e:
        raise RuntimeError(f"No se pudo abrir el puerto {port}: {e}") from e

    try:
        # Dar tiempo para que el dispositivo se estabilice
        time.sleep(0.1)
        payload = (state + "\n").encode()
        ser.write(payload)
        # Intentar leer una línea de respuesta (si el ESP32 responde)
        try:
            resp = ser.readline().decode(errors='ignore').strip()
        except Exception:
            resp = None
        print(f"Enviado: '{state}' -> {port} ; Respuesta: '{resp}'")
        return resp
    finally:
        ser.close()


# Función rápida que detecta puerto y envía el estado dado (útil para pruebas)
def enviar_estado_auto(state: str, preferido: str = None):
    port = seleccionar_puerto(preferido)
    if port is None:
        raise RuntimeError('No hay puertos serial disponibles para enviar el estado.')
    return send_state(port, state)


print('Celda de serial cargada. Ejecuta listar_puertos() para ver los puertos.')
print('Usar: seleccionar_puerto() o enviar_estado_auto("sana") para pruebas.')


In [ ]:
# --- Enviar resultado final al ESP32 (usa funciones serial ya definidas)
# Esta celda toma 'results_list' o 'results_df', calcula la clase con mayor promedio de confianza
# y manda el estado mapeado ('sana','danada','defectosa') al ESP32 usando enviar_estado_auto o send_state.

# Mapeo por defecto: ajusta según tus clases reales
CLASS_TO_STATE = {
    'person': 'danada',
    'none': 'sana',
    'sana': 'sana',
    'defect': 'defectosa',
    'defectosa': 'defectosa',
    'defectuosa': 'defectosa'
}

# Si ya sabes el puerto y la placa, configúralos aquí para pruebas automáticas
PREFERRED_PORT = '/dev/ttyUSB0'  # cambia si tu puerto es distinto
BOARD_NAME = 'ESP32 Dev Module'  # nombre de tu placa (solo informativo)

print(f"Usando puerto preferido: {PREFERRED_PORT} ; Board: {BOARD_NAME}")


def decide_and_send():
    import time, traceback
    # Determinar final_class/final_conf a partir de results_list o results_df
    try:
        if 'results_df' in globals() and not results_df.empty:
            # Construir promedios por clase a partir de results_list (si existe)
            class_scores = {}
            for r in results_list:
                cls = (r.get('class') or 'None')
                class_scores.setdefault(cls, []).append(r.get('conf', 0.0))
            avg_class_scores = {k: (sum(v)/len(v) if len(v)>0 else 0.0) for k,v in class_scores.items()}
            final_class = max(avg_class_scores.items(), key=lambda x: x[1])[0] if len(avg_class_scores)>0 else 'None'
            final_conf = avg_class_scores.get(final_class, 0.0)
        elif 'final_class' in globals():
            final_class = final_class
            final_conf = globals().get('final_conf', 0.0)
        else:
            raise RuntimeError('No hay resultados para decidir. Ejecuta las celdas de captura y predicción primero.')

        print('Clase final decidida:', final_class, 'conf:', final_conf)
        key = str(final_class).lower() if final_class is not None else 'none'
        state = CLASS_TO_STATE.get(key)
        if state is None:
            # Si la clase no está mapeada: si es 'None' -> sana, else -> danada
            if key in ('none','nan',''):
                state = 'sana'
            else:
                state = 'danada'
        print('Estado que se enviará:', state)

        # Intentar enviar usando la función de mayor nivel primero (enviar_estado_auto), si falla hacer fallback
        resp = None
        sent = False

        # 1) Intento directo con enviar_estado_auto (si existe)
        if 'enviar_estado_auto' in globals():
            try:
                print('Intentando enviar con enviar_estado_auto(...)')
                # pasar el puerto preferido para forzar uso de /dev/ttyUSB0 si está conectado
                resp = enviar_estado_auto(state, preferido=PREFERRED_PORT)
                print('Respuesta enviar_estado_auto:', resp)
                sent = True
            except Exception as e:
                print('Error en enviar_estado_auto:', e)
                traceback.print_exc()
                sent = False

        # 2) Fallback: intentar seleccionar puerto y usar send_state
        if not sent:
            try:
                print('FALLBACK: intentando enviar con send_state(...)')
                port = None
                if 'seleccionar_puerto' in globals():
                    try:
                        port = seleccionar_puerto(PREFERRED_PORT)
                        print('Puerto seleccionado por seleccionar_puerto():', port)
                    except Exception as e:
                        print('seleccionar_puerto falló:', e)
                        port = None

                # si aún no hay puerto, intentar listar puertos y escoger manualmente
                if port is None and 'listar_puertos' in globals():
                    try:
                        ports = listar_puertos()
                        print('Puertos detectados:', ports)
                        if PREFERRED_PORT in ports:
                            port = PREFERRED_PORT
                        elif ports:
                            port = ports[0]
                    except Exception as e:
                        print('listar_puertos falló:', e)
                        port = None

                if port is None:
                    raise RuntimeError('No se encontró ningún puerto serie disponible para enviar el estado.')

                # usar baudrate si está definido en el kernel (ej.: variable global 'baudrate')
                baud_to_use = globals().get('baudrate', 115200)
                resp = send_state(port, state, baud=baud_to_use)
                print('Respuesta send_state:', resp)
                sent = True
            except Exception as e:
                print('Error en fallback send_state:', e)
                traceback.print_exc()
                sent = False

        if not sent:
            print('No fue posible enviar el estado. Revisa conexión serie y permisos.')

        return state, resp
    except Exception as e:
        print('Error en decide_and_send:')
        traceback.print_exc()
        raise


# Ejecutar la decisión y envío una sola vez (esto no entra al bucle de vídeo)
print('Ejecutando decide_and_send() ...')
try:
    estado_enviado, respuesta = decide_and_send()
    print(f'Estado enviado: {estado_enviado} ; Respuesta: {respuesta}')
except Exception as e:
    print('No se pudo enviar el estado automáticamente:', e)
